In [1]:
from glob import glob
import sys
import os
# import psutil
from os.path import join, dirname
from os.path import getsize
import pickle
import numpy as np
import pandas as pd
import scipy.stats

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.multitest

# import nilearn.masking
from nilearn import plotting as nplt
from nilearn import image as niimg
import nilearn.decoding

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC

import mydef_GA

%matplotlib inline

In [2]:
## LDA analysis
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')

In [3]:
from datetime import date
today = date.today().strftime("%Y%m%d")

In [4]:
root_dir = mydef_GA.root_dir

script_dir = mydef_GA.script_dir

fmri_dir = mydef_GA.fmri_dir
LSS_dir = mydef_GA.LSS_dir
mask_dir = mydef_GA.mask_dir
dmn_dir = mydef_GA.dmn_dir
loc_dir = mydef_GA.loc_dir

In [5]:
subj_list = mydef_GA.subj_list
stage_list = mydef_GA.stage_list

In [6]:
## labeling with target position
# 1 - 5 - 25 - 21 - 1 - 25 - 5 - 21 - 25 - 1 - 21 - 5 - 1 - ...
##################
#  1  2  3  4  5 #
#  6  7  8  9 10 #
# 11 12 13 14 15 #
# 16 17 18 19 20 #
# 21 22 23 24 25 #
##################
target_pos = []

# with open(join(root_dir,'targetID.txt')) as file:
with open(join(script_dir,'targetID.txt')) as file:
    for line in file:
        target_pos.append(int(line.strip()))
        
target_pos = target_pos[1:97]
# target_path = list(range(1,13))*8

In [7]:
## background image
img_bg = join(mask_dir,'mni152_2009bet.nii.gz')

# ROI list

### Movement-related ROIs from localizer

In [8]:
## ROI
roi_imgs = {}
path_list = sorted(glob(join(mask_dir,'localizer','n200_*.nii')))
for path in path_list:
    roi = path.split('/')[-1].replace('_mask.nii', '')
    roi_imgs[roi] = niimg.load_img(path)
roi_imgs

{'n200_c1-1_L_M1': <nibabel.nifti1.Nifti1Image at 0x7fb1508c9e50>,
 'n200_c1-2_L_S1': <nibabel.nifti1.Nifti1Image at 0x7fb140efd310>,
 'n200_c1_L_Postcentral': <nibabel.nifti1.Nifti1Image at 0x7fb140efd670>,
 'n200_c1_R_SPL': <nibabel.nifti1.Nifti1Image at 0x7fb140efda30>,
 'n200_c2_R_CerebellumIV-V': <nibabel.nifti1.Nifti1Image at 0x7fb140efdbb0>,
 'n200_c2_R_MFG': <nibabel.nifti1.Nifti1Image at 0x7fb1508c9220>,
 'n200_c3_R_MTG': <nibabel.nifti1.Nifti1Image at 0x7fb140efd8e0>,
 'n200_c3_R_Postcentral': <nibabel.nifti1.Nifti1Image at 0x7fb140efd910>,
 'n200_c4_L_IFG': <nibabel.nifti1.Nifti1Image at 0x7fb140f09100>,
 'n200_c4_L_Putamen': <nibabel.nifti1.Nifti1Image at 0x7fb140f095b0>,
 'n200_c5_L_SPL': <nibabel.nifti1.Nifti1Image at 0x7fb140f09970>,
 'n200_c5_R_SMA': <nibabel.nifti1.Nifti1Image at 0x7fb140f09d00>,
 'n200_c6_R_CerebellumVIIIb': <nibabel.nifti1.Nifti1Image at 0x7fb140f09be0>,
 'n200_c6_R_MFG': <nibabel.nifti1.Nifti1Image at 0x7fb140f0c430>,
 'n200_c7_L_Thalamus': <nibabel

### Default Mode Network

In [9]:
roi_imgs = {}
roi_names = []

## DMN regions(global)
module_names = ['Core', 'dMsub', 'MTLsub']

for name in module_names:
    DMN_list = glob(join(dmn_dir, name+'.nii'))
    for path in DMN_list:
        fname = path.split('/')[-1].replace('.nii', '')
        roi_imgs[fname] = nilearn.image.load_img(path)
        roi_names.append(fname)
        
# ## DMN regions(detail)
module_names = ['Core', 'dMsub', 'MTLsub']

for name in module_names:
    DMN_list = glob(join(dmn_dir, name+'_*.nii'))
    for path in DMN_list:
        fname = path.split('/')[-1].replace('.nii', '')
        roi_imgs[fname] = nilearn.image.load_img(path)
        roi_names.append(fname)

roi_imgs

{'Core': <nibabel.nifti1.Nifti1Image at 0x7fb1508c9e20>,
 'dMsub': <nibabel.nifti1.Nifti1Image at 0x7fb140f282e0>,
 'MTLsub': <nibabel.nifti1.Nifti1Image at 0x7fb140f28400>,
 'Core_PCC_l': <nibabel.nifti1.Nifti1Image at 0x7fb140f286a0>,
 'Core_PCC_r': <nibabel.nifti1.Nifti1Image at 0x7fb140f28a60>,
 'Core_aMPFC_l': <nibabel.nifti1.Nifti1Image at 0x7fb140f28e20>,
 'Core_aMPFC_r': <nibabel.nifti1.Nifti1Image at 0x7fb140f19280>,
 'dMsub_LTC_l': <nibabel.nifti1.Nifti1Image at 0x7fb140f19610>,
 'dMsub_LTC_r': <nibabel.nifti1.Nifti1Image at 0x7fb140f19a30>,
 'dMsub_TPJ_l': <nibabel.nifti1.Nifti1Image at 0x7fb140f195e0>,
 'dMsub_TPJ_r': <nibabel.nifti1.Nifti1Image at 0x7fb140f24160>,
 'dMsub_TempP_l_temp': <nibabel.nifti1.Nifti1Image at 0x7fb140f24520>,
 'dMsub_TempP_r_temp': <nibabel.nifti1.Nifti1Image at 0x7fb140f28340>,
 'dMsub_dMPFC': <nibabel.nifti1.Nifti1Image at 0x7fb140f283d0>,
 'MTLsub_HF_l': <nibabel.nifti1.Nifti1Image at 0x7fb140f24130>,
 'MTLsub_HF_r': <nibabel.nifti1.Nifti1Image 

### Ventral visual stream (TT_Daemon, Brodmann Area)

In [10]:
roi_imgs = {}
path_list = glob(join(mask_dir, 'TT_Daemon', '*.brik1.BA???.*.nii.gz'))
for path in path_list:
    temp = path.split('/')[-1].replace('.nii.gz', '')
    fname = temp.split('.')[2]
    roi_imgs[fname] = nilearn.image.load_img(path)
roi_imgs

{'BA17L': <nibabel.nifti1.Nifti1Image at 0x7fb140f0f640>,
 'BA17R': <nibabel.nifti1.Nifti1Image at 0x7fb140f0f490>,
 'BA18L': <nibabel.nifti1.Nifti1Image at 0x7fb140f0fbe0>,
 'BA18R': <nibabel.nifti1.Nifti1Image at 0x7fb140f32a30>,
 'BA19L': <nibabel.nifti1.Nifti1Image at 0x7fb140f32d30>,
 'BA19R': <nibabel.nifti1.Nifti1Image at 0x7fb140f37370>,
 'BA20L': <nibabel.nifti1.Nifti1Image at 0x7fb140f37700>,
 'BA20R': <nibabel.nifti1.Nifti1Image at 0x7fb140f37a90>,
 'BA37L': <nibabel.nifti1.Nifti1Image at 0x7fb140f37eb0>,
 'BA37R': <nibabel.nifti1.Nifti1Image at 0x7fb140f3f070>,
 'BA38L': <nibabel.nifti1.Nifti1Image at 0x7fb140f3f580>,
 'BA38R': <nibabel.nifti1.Nifti1Image at 0x7fb140f3f910>}

### Fan280

In [11]:
fan_imgs={}
path_list = glob(join(mask_dir,'fan280','*.nii.gz'))
for path in path_list:
    temp = path.split('/')[-1].replace('.nii.gz', '')
    fname = temp.split('.')[-1]
    fan_imgs[fname] = nilearn.image.load_img(path)

# Encoded activity patterns

In [12]:
subj = '01'
stage = 'early_practice'
beta = mydef_GA.load_betas(subj, stage)

In [ ]:
_, axes = plt.subplots(2,2,figsize=(20,20))
axes = np.concatenate(axes)

# subj = random.sample(range(30),k=1)[0]
# print('subj : GA%s'%nn_list[subj])
cnt = 0
for mapping in ['practiced', 'unpracticed']:
    for stage in ['Early', 'Late']:
        corr_mat = np.ones((len(rois),len(rois)))
        for i, ff in enumerate(rois):
            for j, tt in enumerate(rois[(i+1):]):
                corr_mat[i,j+i+1] = np.mean(scores[stage,mapping,(*sorted((ff,tt)),)])
#                 corr_mat[i,j+i+1] = scores[stage,mapping,(*sorted((ff,tt)),)][subj]
                corr_mat[j+i+1,i] = corr_mat[i,j+i+1]
        nplt.plot_matrix(corr_mat, labels=rois
                         , colorbar=False, auto_fit=True
                         , vmax=1., vmin=-1., axes=axes[cnt]
                         , title="%s / %s"%(stage,mapping)
                        )
        cnt += 1